this report generates all plots for ablation study.


there are four sets of data to read

1. original recurrent models, 2L, 16/32 ch, average depth as well as performance.
2. multipath models, 2L, 16/32 ch, average depth as well as performance.
3. ablated multipath models, onlyDX, 2L, 16/32 ch, average depth as well as performance.
4. ablated multipath models, leDX/geDX, 2L, 16/32 ch, average depth as well as performance.

I should denote them with different colors.


I should plot data at two levels.

1. overall level
2. per read out type. this is to show that 8K data does not work well with "single path does not work" hypothesis


storage format.

a big dataframe

* `data_source`: 'original', 'multipath', 'onlyDX'. 'leDXgeDX'
* `aggregate_level`: '', 'readout_type=inst-avg', 'readout_type=inst-last', 'readout_type=cm-avg', 'readout_type=cm-last'
    * '' means everything
    * more general, it's `/` joined by `k=v`
* `rcnn_bl_cls`: 2,3,4,5,6,7
    * no feed-forward models involved
* `n`: how many raw models are involved to get every number that follows
* `depth_scalar_mean`: a list of depths.
    * for `leDX/geDX`, this will be a proper list, with 2*`rcnn_bl_cls` numbers, following the order in plots in `/results_processed/tang_refactored/maskcnn_polished_with_rcnn_k_bl/20201218_plot.ipynb`
    * for others, this will be a list with a single number.
* `depth_scalar_sem`: similar to above, but sem. calculated with `sem(ddof=0)`.
* `cc2_normed_avg_mean`: similar to `depth_scalar_mean`
* `cc2_normed_avg_sem`: similar to `depth_scalar_sem`


Note that, for every combination of (`data_source`, `aggregate_level`, `rcnn_bl_cls`), the `n` should be the same and only and all models that are available in `multipath` should be analzyed. This is because `multipath` models have the most demanding requirement on memory so there are fewest of them.

In [1]:
from os.path import join, dirname, exists
from os import makedirs
from thesis_v2 import dir_dict

In [2]:
import pandas as pd
import numpy as np

In [3]:
from thesis_v2.analysis.ablation_study import collect_all_data

In [4]:
def load_original_perf():
    # this does not contain 48, 64 channel results, though.
    # (I forgot to train these models in ensemble mode)
    f_main_result_perf = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200725+20200801+20200801_2', 'main_result.pkl')
    df_main_result_perf = pd.read_pickle(f_main_result_perf).sort_index()
    
    return df_main_result_perf

In [5]:
def load_original_depth():
    # this does not contain 48, 64 channel results, though.
    # (I forgot to train these models in ensemble mode)
    f_main_result = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20200725+20200801+20200801_2', 'source_analysis.pkl')
    df_main_result = pd.read_pickle(f_main_result).sort_index()
    return df_main_result

In [6]:
def load_multipath_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201118', 'main_result_separatebn.pkl')),
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201114', 'main_result_separatebn.pkl')),
        ], axis=0
    )
    return ret

def load_multipath_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201114+20201118_separatebn', 'source_analysis.pkl'))
    return ret

In [7]:
def load_onlyDX_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201221', 'main.pkl')),
        ], axis=0
    )
    return ret

In [8]:
def load_onlyDX_depth():
    ret = pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201221', 'source_analysis.pkl'))
    return ret

In [9]:
def load_leDXgeDX_perf():
    ret = pd.concat(
        [
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201205+20201205_2', 'main.pkl')),
            pd.read_pickle(join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201213+20201213_2', 'main.pkl')),    
        ], axis=0
    )
    return ret

In [10]:
def load_leDXgeDX_depth():
    f_main_result_src = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201205+20201205_2', 'source_analysis.pkl')
    f_main_result_2_src = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', '20201213+20201213_2', 'source_analysis.pkl')
    
    df_main_result_src = pd.read_pickle(f_main_result_src)
    df_main_result_2_src = pd.read_pickle(f_main_result_2_src)
    
    df_main_result_src = pd.concat([df_main_result_src, df_main_result_2_src], axis=0).sort_index()
    assert df_main_result_src.index.is_unique
    
    return df_main_result_src

In [12]:
f_main_result = join(dir_dict['analyses'], 'yuanyuan_8k_a_3day_refactored', 'ablation_2L_32ch_5120.pkl')
makedirs(dirname(f_main_result), exist_ok=True)

if not exists(f_main_result):
    df_main_result = collect_all_data(
    data_loader_dict = {
        'multipath': {
            'perf': load_multipath_perf,
            'depth': load_multipath_depth,
        },
        'original': {
            'perf': load_original_perf,
            'depth': load_original_depth,
        },
        'onlyDX': {
            'perf': load_onlyDX_perf,
            'depth': load_onlyDX_depth,
        },
        'leDXgeDX': {
            'perf': load_leDXgeDX_perf,
            'depth': load_leDXgeDX_depth,
        }
    },
        train_keep=5120,
        out_channel=(32,),
    )
    df_main_result.to_pickle(f_main_result)
    del df_main_result
df_main_result = pd.read_pickle(f_main_result)

(352,)


In [13]:
df_main_result

depth_mean  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                                                   [0.0, 1.0]   
                                   3            [0.0, 0.6659736644960162, 1.0636163926538083, ...   
                                   4            [0.0, 0.660282250061316, 1.115993401134694, 1....   
                                   5            [0.0, 0.6700893364664616, 1.0381003094109462, ...   
                                   6            [0.0, 0.6739658492171491, 1.057441830369511, 1...   
...                                                                                           ...   
original    readout_type=inst-last 3                                         [1.5785852528184878]   
                                   4                                          [2.245080713477078]   
                                   5                                          [2.830659201400932]   
                                   6                                          [3.312625002949721]   
                                   7                                         [2.5422780559621057]   

                                                                                        depth_sem  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                                                   [0.0, 0.0]   
                                   3            [0.0, 0.015307060639857002, 0.0029534231599443...   
                                   4            [0.0, 0.01263975376188505, 0.03006171688646959...   
                                   5            [0.0, 0.01187903062335674, 0.0210386067684209,...   
                                   6            [0.0, 0.012007834216373242, 0.0207365463416972...   
...                                                                                           ...   
original    readout_type=inst-last 3                                       [0.018284187407086316]   
                                   4                                        [0.05318308898619729]   
                                   5                                        [0.08533864082098758]   
                                   6                                        [0.10013069874389213]   
                                   7                                        [0.15691721497713215]   

                                                 n  \
data_source aggregate_level        rcnn_bl_cls       
leDXgeDX                           2            64   
                                   3            64   
                                   4            64   
                                   5            64   
                                   6            64   
...                                             ..   
original    readout_type=inst-last 3            16   
                                   4            16   
                                   5            16   
                                   6            16   
                                   7             8   

                                                                                        perf_mean  \
data_source aggregate_level        rcnn_bl_cls                                                      
leDXgeDX                           2                      [0.587720271335443, 0.6428338917151981]   
                                   3            [0.5871894000138049, 0.6434333593227827, 0.655...   
                                   4            [0.5877250725441736, 0.644935275961269, 0.6553...   
                                   5            [0.587380604376706, 0.6469099446630671, 0.6580...   
                                   6            [0.587446177210466, 0.646588854409776, 0.65796...   
...                                              